In [1]:
import os

In [2]:
%pwd

'c:\\Users\\examination8550\\notebook\\chicken-disease-classification\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\examination8550\\notebook\\chicken-disease-classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
Path('artifacts/prepare_callbacks/checkpoint_dir/model.h5')

WindowsPath('artifacts/prepare_callbacks/checkpoint_dir/model.h5')

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import logger
class ConfigurationManager:
    def __init__(self,
                  config_filepath = CONFIG_FILE_PATH,
                  params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath) 
        create_directories([self.config.artifacts_root])
        logger.info(f'created directory at: {self.config.artifacts_root}')
    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir), 
                            Path(ckpt_dir)])
        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, 
                                          f'tb_logs_at_{timestamp}')
        return tf.keras.callbacks.TensorBoard(tb_running_log_dir)
    @property
    def _create_chkpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=self.config.checkpoint_model_filepath,
                                                   save_best_only=True)
    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks, self._create_chkpt_callbacks]


In [13]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callbacks_config()
    callbacks = PrepareCallbacks(prepare_callbacks_config)
    callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2023-06-11 18:11:33,013: INFO: common: yaml file: config\config.yaml is loaded succesfully.]
[2023-06-11 18:11:33,039: INFO: common: yaml file: params.yaml is loaded succesfully.]
[2023-06-11 18:11:33,056: INFO: 1885244473: created directory at: artifacts]
[2023-06-11 18:11:33,067: INFO: common: Created a directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2023-06-11 18:11:33,071: INFO: common: Created a directory at: artifacts\prepare_callbacks\checkpoint_dir]
